In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import skimage
import tensorflow as tf
import scipy

In [9]:
import inception_base

## Initialize Constants

In [3]:
train_val_video_path = "..\\SURA\\dataset\\raw\\MSR-VTT_2016\\TrainValVideo\\"
test_video_path = "..\\SURA\\dataset\\raw\\MSR-VTT_2016\\TestVideo/"
base_save_path = ".\\"
num_frames_per_video = 100
vid_filter = lambda x: x.endswith('mp4')
train_filter =  lambda x: 0 <= int(x[5:][:-4]) <= 6512
val_filter =  lambda x: 6513 <= int(x[5:][:-4]) <= 7009
test_filter =  lambda x: 7010 <= int(x[5:][:-4]) <= 9999

In [4]:
videos = os.listdir(train_val_video_path) +  os.listdir(test_video_path)
videos = list(filter(vid_filter,videos))

## Inception Model Declare and restore Weights``

In [5]:
class Inception_Model(object):
    def __init__(self,learning_rate=1e-3,initializer_scale=0.5):     
        self.learning_rate = learning_rate
        self.initializer = tf.random_uniform_initializer(
                                            minval=-initializer_scale,
                                            maxval=initializer_scale)
        self.images = None
        # Global step Tensor.
        self.global_step = None
    def build_inputs_outputs(self):
        self.image_feed = tf.placeholder(dtype=tf.float32,shape=[None,299,299,3],name="image_feed")        
    def build_image_embedding(self):
        self.inception_output = inception_base.get_base_model(self.image_feed)
        self.fine_tune_input = tf.placeholder_with_default(self.inception_output,self.inception_output.get_shape(),name="fine_tune_input")
        self.inception_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="InceptionV4")
#     def setup_global_step(self):
#         """Sets up the global step Tensor."""
#         global_step = tf.Variable(
#             initial_value=0,
#             name="global_step",
#             trainable=False,
#             collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
#         self.global_step = global_step
        
    def build(self):
        self.build_inputs_outputs()
        self.build_image_embedding()

In [6]:
model = Inception_Model()
model.build()

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
saver = tf.train.Saver(var_list=model.inception_variables)
saver.restore(sess,"./inception_v4.ckpt")

INFO:tensorflow:Restoring parameters from ./inception_v4.ckpt


In [ ]:
#out = sess.run(model.inception_output,feed_dict={model.image_feed:IMAGE_BATCH_ARRAY)})

## Main Function to read video, trim frames, and preprocess images

In [9]:
def read_video(video_path):
    #p = os.path.join(train_val_video_path+x)
    cap  = cv2.VideoCapture( video_path )
    frame_count = 0
    frame_list = []
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame_list.append(frame)
        frame_count += 1
    frame_list = np.array(frame_list)
    if frame_count > num_frames_per_video:
            frame_indices = np.linspace(0, frame_count-1, num=num_frames_per_video, endpoint=True).astype(int)
            frame_list = frame_list[frame_indices]
    frame_list = [cv2.cvtColor(x, cv2.COLOR_BGR2RGB) for x in frame_list]
    frame_list_resized = [scipy.misc.imresize(frame,[299,299,3]) for frame in frame_list]
    frame_list_processed = [((2*(frame.astype(np.float32) / 255 ))-1) for frame in frame_list_resized]
    frame_list = np.array(frame_list_processed)
    return frame_list

In [2]:
import numpy as np

In [4]:
np.linspace(0,3,5,endpoint=False)

array([ 0. ,  0.6,  1.2,  1.8,  2.4])

In [6]:
np.ones([5],dtype=np.uint8)

array([1, 1, 1, 1, 1], dtype=uint8)

In [11]:
z=  [1]*5+[0]*3

In [14]:
np.asarray(z,dtype=np.uint8)

array([1, 1, 1, 1, 1, 0, 0, 0], dtype=uint8)

In [9]:
z[0] = 2

In [3]:
import cv2

In [22]:
# cap  = cv2.VideoCapture("/home/ozym4nd145/Coding/Notebook/SURA/dataset/raw/MSR-VTT_2016/TrainValVideo/video1648.mp4")
cap  = cv2.VideoCapture("../dataset/raw/MSR-VTT_2016/TrainValVideo/video16.mp4")
frame_count = 0
frame_list = []
while True:
    ret, frame = cap.read()
    if ret is False:
        break
    frame_list.append(frame)
    frame_count += 1

In [ ]:
cv2.VideoCapture()

In [23]:
frame_list

[]

### Train, Val and Test 

In [13]:
count = 1
for vid in  filter(train_filter,videos):
    read_path = os.path.join(train_val_video_path,vid)
    save_path = os.path.join(base_save_path,"train",vid+".npy")
    if(not os.path.exists(save_path)):
        preprocessed_frames = read_video(read_path)
        completed_frames = sess.run(model.inception_output,feed_dict={model.image_feed:preprocessed_frames})
        np.save(save_path,completed_frames)
        if count%100 == 0:
            print("Processed 100 videos, currently at "+vid+"\n")
        count+=1

Processed 100 videos, currently at video1087.mp4

Processed 100 videos, currently at video1177.mp4

Processed 100 videos, currently at video1267.mp4

Processed 100 videos, currently at video1357.mp4

Processed 100 videos, currently at video1447.mp4

Processed 100 videos, currently at video1537.mp4

Processed 100 videos, currently at video1627.mp4

Processed 100 videos, currently at video1717.mp4

Processed 100 videos, currently at video1807.mp4

Processed 100 videos, currently at video1898.mp4

Processed 100 videos, currently at video1988.mp4

Processed 100 videos, currently at video2077.mp4

Processed 100 videos, currently at video2167.mp4

Processed 100 videos, currently at video2257.mp4

Processed 100 videos, currently at video2347.mp4

Processed 100 videos, currently at video2437.mp4

Processed 100 videos, currently at video2527.mp4

Processed 100 videos, currently at video2617.mp4

Processed 100 videos, currently at video2707.mp4

Processed 100 videos, currently at video2798.mp4



In [15]:
count = 1
for vid in  filter(test_filter,videos):
    read_path = os.path.join(test_video_path,vid)
    save_path = os.path.join(base_save_path,"test",vid+".npy")
    if(not os.path.exists(save_path)):
        preprocessed_frames = read_video(read_path)
        completed_frames = sess.run(model.inception_output,feed_dict={model.image_feed:preprocessed_frames})
        np.save(save_path,completed_frames)
        if count%100 == 0:
            print("Processed 100 videos, currently at "+vid+"\n")
        count+=1

Processed 100 videos, currently at video7109.mp4

Processed 100 videos, currently at video7209.mp4

Processed 100 videos, currently at video7309.mp4

Processed 100 videos, currently at video7409.mp4

Processed 100 videos, currently at video7509.mp4

Processed 100 videos, currently at video7609.mp4

Processed 100 videos, currently at video7709.mp4

Processed 100 videos, currently at video7809.mp4

Processed 100 videos, currently at video7909.mp4

Processed 100 videos, currently at video8009.mp4

Processed 100 videos, currently at video8109.mp4

Processed 100 videos, currently at video8209.mp4

Processed 100 videos, currently at video8309.mp4

Processed 100 videos, currently at video8409.mp4

Processed 100 videos, currently at video8509.mp4

Processed 100 videos, currently at video8609.mp4

Processed 100 videos, currently at video8709.mp4

Processed 100 videos, currently at video8809.mp4

Processed 100 videos, currently at video8909.mp4

Processed 100 videos, currently at video9009.mp4



In [16]:
count = 1
for vid in  filter(val_filter,videos):
    read_path = os.path.join(train_val_video_path,vid)
    save_path = os.path.join(base_save_path,"val",vid+".npy")
    if(not os.path.exists(save_path)):
        preprocessed_frames = read_video(read_path)
        completed_frames = sess.run(model.inception_output,feed_dict={model.image_feed:preprocessed_frames})
        np.save(save_path,completed_frames)
        if count%100 == 0:
            print("Processed 100 videos, currently at "+vid+"\n")
        count+=1

Processed 100 videos, currently at video6612.mp4

Processed 100 videos, currently at video6712.mp4

Processed 100 videos, currently at video6812.mp4

Processed 100 videos, currently at video6912.mp4



In [ ]:
preprocessed_frames = read_video(read_path)

In [ ]:
preprocessed_frames[8][77]

### Cv2 gives height, width, channel

In [ ]:
cv2.imshow('image',frame_list[0])
cv2.waitKey(0)

In [ ]:
x.shape

In [ ]:
x = frame_list[0]

In [ ]:
d = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(preprocessed_frames[7])

In [ ]:
plt.show()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(d)

In [ ]:
plt.show()

In [ ]:
frame_list = np.array(frame_list)

In [ ]:
if frame_count > num_frames_per_video:
            frame_indices = np.linspace(0, frame_count-1, num=num_frames_per_video, endpoint=True).astype(int)
            frame_list = frame_list[frame_indices]

In [ ]:
frame_list.shape

In [ ]:
def 

In [ ]:
t = scipy.ndimage.imread("12447.jpg")

In [ ]:
t.shape

In [ ]:
t = scipy.misc.imresize(d,[300,300,3])

In [2]:
import json
import os
import configuration
from data_generator import Data_Generator


In [3]:
data_config = configuration.DataConfig().config
data_gen = Data_Generator(data_config["processed_video_dir"],
                        data_config["caption_file"],
                        data_config["unique_frequency_cutoff"],
                        data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])




In [4]:
ref = {'info':{},'images':[],'licenses':[],'type':'captions','annotations':[]}

ref['info'] = {'contributor': 'Suyash Agrawal',
               'date_created': '2017-06-08',
               'description': 'test',
               'url': 'https://github.com/ozym4nd145',
               'version': '1.0',
               'year': 2017}

ref['licenses'].append({'id': 1,
                        'name': 'Attribution-NonCommercial-ShareAlike License',
                        'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/'})

video_to_id = {}
id_to_video = {}

all_data = []

for i in data_gen.dataset.keys():
    all_data += data_gen.dataset[i]


In [7]:
all_data[0]

{'caption': 'a woman is adding and stirring in three different ingredients into a bowl of cooked white rice',
 'caption_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int32),
 'file_name': '0lh_UWF9ZP4_50_60.avi.npy',
 'indexed_caption': array([  1,   4,   8,   5, 204,  13, 221,   9, 121, 719, 233,  18,   4,
         57,  14, 371, 144, 130,   2,   0,   0], dtype=int32),
 'path': '../dataset/MSVD_processed/val/0lh_UWF9ZP4_50_60.avi.npy',
 'processed_caption': '<bos> a woman is adding and stirring in three different ingredients into a bowl of cooked white rice <eos> <pad> <pad>',
 'real_caption': 'a woman is adding and stirring in three different ingredients into a bowl of cooked white rice.'}

In [8]:

video_ids = list(set([i['file_name'][:-4] for i in all_data]))

video_ids.sort()


In [9]:
for idx,name in enumerate(video_ids):
    dictionary = {}
    dictionary['file_name'] = name[:-4]
    dictionary['id'] = idx
    dictionary['license'] = 1
    ref['images'].append(dictionary)
    video_to_id[name[:-4]] = idx
    id_to_video[idx] = name[:-4]


In [12]:
all_data.sort(key= lambda x: (x["file_name"]))
